In [14]:
pip install crewai

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached aiohttp-3.12.13-cp311-cp311-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached tiktoken-0.9.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached orjson-3.10.18-cp311-cp311-macosx_15_0_arm64.whl.metadata (41 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp31

In [ ]:
import crewai as crew
import vision_parse
from vision_parse import VisionParser, PDFPageConfig
from crewai import Agent, Task, Crew, Process, LLM

llm=LLM(model="ollama/llama3.2", 
        base_url="http://localhost:11434")

def pdf_to_markdown(pdf_path: str) -> str:
    parser = VisionParser(
        model_name="llama3.2-vision:11b", 
        temperature=0.4,
        top_p=0.5,
        image_mode="url", 
        detailed_extraction=False, 
        enable_concurrency=False,
    )

    markdown_pages = parser.convert_pdf(pdf_path)
    result = ""
    for i, page_content in enumerate(markdown_pages):
        print(f"\n--- Page {i+1} ---\n{page_content}")
        result += f"\n--- Page {i+1} ---\n{page_content}\n"
    return result

splitwise_agent = Agent(
    role="An intelligent financial assistant skilled at splitting bills fairly based on item ownership and tax allocation.",
    goal=(
        "To accurately divide the total bill among participants by identifying who bought which items, "
        "calculating each person's share including tax, and providing clear summaries of amounts owed."
        "assign the portion of the bill to each person based on the items they purchased."
    ),
    backstory=(
        "I was created to make group bill splitting effortless, especially in scenarios where different people "
        "purchase different items. I carefully factor in taxes and ensure that each participant pays exactly for "
        "what they bought plus their fair share of taxes, so the process is transparent and fair."
    ),
    llm=llm
)

spitwise_task = Task(
    description="""
        Split the total bill among a group of people according to who purchased each item.
        Make sure to include tax proportionally for each item.
        Provide a clear breakdown of who owes what.
    """,
    expected_output="""
        A detailed summary showing each person's total amount owed, including tax, based on their purchased items.
    """,
    agent=splitwise_agent
)

crew=Crew(
    agents=[splitwise_agent],
    tasks=[spitwise_task],
    process=Process.sequential

)

bill_path="bill.pdf"
Markdown_content= pdf_to_markdown(bill_path)

inputs={
    "bill_content": Markdown_content,
    "details": {
        "Jay":["eggs","milk","apples"], 
        "Bob":["carrots","balloons"],
        "christina":["oranges"]
}}

result=crew.run(inputs=inputs)

print(result)


Converting pages in PDF file into markdown format:   0%|          | 0/1 [00:01<?, ?it/s]


LLMError: Ollama Model processing failed: error starting runner: fork/exec /Applications/Ollama.app/Contents/Resources/ollama: no such file or directory  (status code: 500)

vision-parse                             0.1.13
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys
print(sys.executable)
import crewai as crew
import vision_parse
from vision_parse import VisionParser, PDFPageConfig
from crewai import Agent, Task, Crew, Process, LLM

llm = LLM(model="ollama/llama3.2", 
          base_url="http://localhost:11434")

def pdf_to_markdown(pdf_path: str) -> str:
    parser = VisionParser(
        model_name="llama3.2-vision:11b", 
        temperature=0.4,
        top_p=0.5,
        image_mode="url", 
        detailed_extraction=False, 
        enable_concurrency=False,
    )

    markdown_pages = parser.convert_pdf(pdf_path)
    result = ""
    for i, page_content in enumerate(markdown_pages):
        print(f"\n--- Page {i+1} ---\n{page_content}")
        result += f"\n--- Page {i+1} ---\n{page_content}\n"
    return result

splitwise_agent = Agent(
    role="An intelligent financial assistant skilled at splitting bills fairly based on item ownership and tax allocation.",
    goal=(
        "To accurately divide the total bill among participants by identifying who bought which items, "
        "calculating each person's share including tax, and providing clear summaries of amounts owed. "
        "Assign the portion of the bill to each person based on the items they purchased."
    ),
    backstory=(
        "I was created to make group bill splitting effortless, especially in scenarios where different people "
        "purchase different items. I carefully factor in taxes and ensure that each participant pays exactly for "
        "what they bought plus their fair share of taxes, so the process is transparent and fair."
    ),
    llm=llm  
)

splitwise_task = Task(  
    description="""
        You are given a bill in markdown format and details about who purchased which items.
        
        Bill Content: {bill_content}
        
        Purchase Details: {details}
        
        Your task is to:
        1. Parse the bill content to identify all items and their prices
        2. Calculate the subtotal, tax amount, and total
        3. Assign each item to the correct person based on the purchase details
        4. Calculate each person's subtotal (sum of their items)
        5. Allocate tax proportionally based on each person's subtotal
        6. Provide a clear breakdown showing:
           - Each person's items and their costs
           - Each person's subtotal
           - Each person's tax portion
           - Each person's total amount owed
        
        Make sure all calculations are accurate and the sum of individual totals equals the bill total.
    """,
    expected_output="""
        A detailed summary showing:
        1. Bill overview (subtotal, tax, total)
        2. For each person:
           - Items purchased and their individual costs
           - Subtotal for their items
           - Tax portion (proportional to their subtotal)
           - Total amount owed
        3. Verification that all individual totals sum to the bill total
    """,
    agent=splitwise_agent
)

crew = Crew(
    agents=[splitwise_agent],
    tasks=[splitwise_task],
    process=Process.sequential
)

# Main execution
if __name__ == "__main__":
    bill_path = "bill.pdf"
    
    try:
        markdown_content = pdf_to_markdown(bill_path)
        
        inputs = {
            "bill_content": markdown_content,
            "details": {
                "Jay": ["eggs", "milk", "apples"], 
                "Bob": ["carrots", "balloons"],
                "Christina": ["oranges"]  
            }
        }
        
        result = crew.kickoff(inputs=inputs) 
        
        print("\n" + "="*50)
        print("BILL SPLITTING RESULT")
        print("="*50)
        print(result)
        
    except Exception as e:
        print(f"Error processing bill: {e}")
        print("Make sure 'bill.pdf' exists and Ollama is running with the llama3.2-vision model.")

/opt/homebrew/opt/python@3.11/bin/python3.11


Converting pages in PDF file into markdown format:   0%|          | 0/1 [00:01<?, ?it/s]


LLMError: Ollama Model processing failed: error starting runner: fork/exec /Applications/Ollama.app/Contents/Resources/ollama: no such file or directory  (status code: 500)